In [38]:
#| hide
#from featsql.core import *

In [39]:
import pandas as pd
from sqlalchemy import create_engine

# featsql

> Create features with sql

This file will become your README and also the index of your documentation.

## Install

```sh
pip install featsql
```

## Configurando a engine

In [40]:
url_db = "sqlite:///../.data/mydatabase.db" 
engine = create_engine(url_db)

## Visão inicial do público

Primeiro vamos observar o formato da tabela spine

In [41]:
df_spine = pd.read_sql("SELECT * FROM tb_spine", engine)
df_spine.head()

,ID,SAFRA_REF,Target
0,4,2023-01-01,0
1,5,2023-01-01,0
2,6,2023-01-01,0
3,7,2023-01-01,0
4,10,2023-01-01,1


## Visão inicial da tabela de variáveis

A tabela de variáveis contém 3 variáveis, duas sendo numéricas e uma categórica. Existem mais ID's únicos e datas disponível nessa tabela do que na tabela spine.

In [42]:
df_data = pd.read_sql("SELECT * FROM tb_feat", engine)
df_data.head()

,ID,SAFRA,FEAT_NUM1,FEAT_NUM2,FEAT_CAT1
0,1,2023-01-01,-17,-17,B
1,2,2023-01-01,-58,-18,C
2,3,2023-01-01,21,63,A
3,4,2023-01-01,-31,36,A
4,5,2023-01-01,-31,-36,A


## Criação de variáveis numéricas

In [43]:
tb_publico='tb_spine'
tb_feat = 'tb_feat'
id = 'ID'
safra_ref = 'SAFRA_REF'
safra = 'SAFRA'
target = 'Target'
feat_num_lista = ['FEAT_NUM1','FEAT_NUM2']
lista_janela = [1,3,6]

In [44]:
query_variaveis_numericas = ""

for i in feat_num_lista:
        query_variaveis_numericas +=f"""
        -- Criação de variáveis numéricas a partir da coluna {i} 
        sum(tb_feat.{i}) AS {i}_SUM_{i}M,
        min(tb_feat.{i}) AS {i}_MIN_{i}M,
        max(tb_feat.{i}) AS {i}_MAX_{i}M,
        avg(tb_feat.{i}) AS {i}_AGV_{i}M,
        """
query_variaveis_numericas = query_variaveis_numericas.rstrip(', \n')
print(query_variaveis_numericas)


        -- Criação de variáveis numéricas a partir da coluna FEAT_NUM1 
        sum(tb_feat.FEAT_NUM1) AS FEAT_NUM1_SUM_FEAT_NUM1M,
        min(tb_feat.FEAT_NUM1) AS FEAT_NUM1_MIN_FEAT_NUM1M,
        max(tb_feat.FEAT_NUM1) AS FEAT_NUM1_MAX_FEAT_NUM1M,
        avg(tb_feat.FEAT_NUM1) AS FEAT_NUM1_AGV_FEAT_NUM1M,
        
        -- Criação de variáveis numéricas a partir da coluna FEAT_NUM2 
        sum(tb_feat.FEAT_NUM2) AS FEAT_NUM2_SUM_FEAT_NUM2M,
        min(tb_feat.FEAT_NUM2) AS FEAT_NUM2_MIN_FEAT_NUM2M,
        max(tb_feat.FEAT_NUM2) AS FEAT_NUM2_MAX_FEAT_NUM2M,
        avg(tb_feat.FEAT_NUM2) AS FEAT_NUM2_AGV_FEAT_NUM2M


In [45]:
# def func_num_feat(i, n):        
#         query_variaveis_numericas=f"""
#         -- Criação de variáveis numéricas a partir da coluna {i} 
#         sum(tb_feat.{i}) AS {i}_SUM_{n}M,
#         min(tb_feat.{i}) AS {i}_MIN_{n}M,
#         max(tb_feat.{i}) AS {i}_MAX_{n}M,
#         avg(tb_feat.{i}) AS {i}_AGV_{n}M
#         """
#         return query_variaveis_numericas
# print(func_num_feat("FEAT_NUM2", 3))

In [46]:
query_janela= ""

for n in lista_janela:
    query_variaveis_numericas=""
    for i in feat_num_lista:
        query_variaveis_numericas +=f"""
             -- Criação de variáveis numéricas a partir da coluna {i} para a janela {n}
            sum(tb_feat.{i}) AS {i}_SUM_{n}M,
            min(tb_feat.{i}) AS {i}_MIN_{n}M,
            max(tb_feat.{i}) AS {i}_MAX_{n}M,
            avg(tb_feat.{i}) AS {i}_AGV_{n}M,
        """
    query_variaveis_numericas = query_variaveis_numericas.rstrip(', \n')

    query_janela += f"""
    -- Criação de variáveis de janela de {n}M
    tb_janela_{n}M as(
        SELECT 
            tb_public.{id},
            tb_public.{safra_ref},
            tb_public.{target},
            {query_variaveis_numericas}

        FROM tb_public
        LEFT JOIN {tb_feat} 
        ON  tb_public.{id} = tb_feat.{id}
        AND (strftime('%Y-%m-%d', date({tb_feat}.{safra}, '+{n} months')) >= tb_public.{safra_ref})
            AND ({tb_feat}.{safra} < tb_public.{safra_ref})
        GROUP BY tb_public.{id}, tb_public.{safra_ref}
),
"""
print(query_janela)


    -- Criação de variáveis de janela de 1M
    tb_janela_1M as(
        SELECT 
            tb_public.ID,
            tb_public.SAFRA_REF,
            tb_public.Target,
            
             -- Criação de variáveis numéricas a partir da coluna FEAT_NUM1 para a janela 1
            sum(tb_feat.FEAT_NUM1) AS FEAT_NUM1_SUM_1M,
            min(tb_feat.FEAT_NUM1) AS FEAT_NUM1_MIN_1M,
            max(tb_feat.FEAT_NUM1) AS FEAT_NUM1_MAX_1M,
            avg(tb_feat.FEAT_NUM1) AS FEAT_NUM1_AGV_1M,
        
             -- Criação de variáveis numéricas a partir da coluna FEAT_NUM2 para a janela 1
            sum(tb_feat.FEAT_NUM2) AS FEAT_NUM2_SUM_1M,
            min(tb_feat.FEAT_NUM2) AS FEAT_NUM2_MIN_1M,
            max(tb_feat.FEAT_NUM2) AS FEAT_NUM2_MAX_1M,
            avg(tb_feat.FEAT_NUM2) AS FEAT_NUM2_AGV_1M

        FROM tb_public
        LEFT JOIN tb_feat 
        ON  tb_public.ID = tb_feat.ID
        AND (strftime('%Y-%m-%d', date(tb_feat.SAFRA, '+1 months')) >= tb_public.SAFRA_

In [47]:
query_join= ""
for i in lista_janela:
    query_join += f"""
    LEFT JOIN tb_janela_{i}M
        ON tb_public.{id} = tb_janela_{i}M.{id}
        AND tb_public.{safra_ref} = tb_janela_{i}M.{safra_ref}
"""
print(query_join)


    LEFT JOIN tb_janela_1M
        ON tb_public.ID = tb_janela_1M.ID
        AND tb_public.SAFRA_REF = tb_janela_1M.SAFRA_REF

    LEFT JOIN tb_janela_3M
        ON tb_public.ID = tb_janela_3M.ID
        AND tb_public.SAFRA_REF = tb_janela_3M.SAFRA_REF

    LEFT JOIN tb_janela_6M
        ON tb_public.ID = tb_janela_6M.ID
        AND tb_public.SAFRA_REF = tb_janela_6M.SAFRA_REF



In [48]:
query_final = f"""
with 
tb_public as (
    SELECT 
        *
    FROM {tb_publico}
),
{query_janela}

tb_join as (
    SELECT 
        *
    FROM tb_public 
    {query_join}
)
    
SELECT 
    *
FROM tb_join
"""

In [49]:
print(query_final)


with 
tb_public as (
    SELECT 
        *
    FROM tb_spine
),

    -- Criação de variáveis de janela de 1M
    tb_janela_1M as(
        SELECT 
            tb_public.ID,
            tb_public.SAFRA_REF,
            tb_public.Target,
            
             -- Criação de variáveis numéricas a partir da coluna FEAT_NUM1 para a janela 1
            sum(tb_feat.FEAT_NUM1) AS FEAT_NUM1_SUM_1M,
            min(tb_feat.FEAT_NUM1) AS FEAT_NUM1_MIN_1M,
            max(tb_feat.FEAT_NUM1) AS FEAT_NUM1_MAX_1M,
            avg(tb_feat.FEAT_NUM1) AS FEAT_NUM1_AGV_1M,
        
             -- Criação de variáveis numéricas a partir da coluna FEAT_NUM2 para a janela 1
            sum(tb_feat.FEAT_NUM2) AS FEAT_NUM2_SUM_1M,
            min(tb_feat.FEAT_NUM2) AS FEAT_NUM2_MIN_1M,
            max(tb_feat.FEAT_NUM2) AS FEAT_NUM2_MAX_1M,
            avg(tb_feat.FEAT_NUM2) AS FEAT_NUM2_AGV_1M

        FROM tb_public
        LEFT JOIN tb_feat 
        ON  tb_public.ID = tb_feat.ID
        AND (strftime(

In [50]:
df_data = pd.read_sql(query_final, engine)
df_data.head()

,ID,SAFRA_REF,Target,ID:1,SAFRA_REF:1,Target:1,FEAT_NUM1_SUM_1M,FEAT_NUM1_MIN_1M,FEAT_NUM1_MAX_1M,FEAT_NUM1_AGV_1M,...,SAFRA_REF:3,Target:3,FEAT_NUM1_SUM_6M,FEAT_NUM1_MIN_6M,FEAT_NUM1_MAX_6M,FEAT_NUM1_AGV_6M,FEAT_NUM2_SUM_6M,FEAT_NUM2_MIN_6M,FEAT_NUM2_MAX_6M,FEAT_NUM2_AGV_6M
0,4,2023-01-01,0,4,2023-01-01,0,NaN,NaN,NaN,NaN,...,2023-01-01,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5,2023-01-01,0,5,2023-01-01,0,NaN,NaN,NaN,NaN,...,2023-01-01,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6,2023-01-01,0,6,2023-01-01,0,NaN,NaN,NaN,NaN,...,2023-01-01,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7,2023-01-01,0,7,2023-01-01,0,NaN,NaN,NaN,NaN,...,2023-01-01,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10,2023-01-01,1,10,2023-01-01,1,NaN,NaN,NaN,NaN,...,2023-01-01,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
